# Solving Linear Systems of Equations using HHL

## Setup
Initialize the IBMQ account for access to IBM quantum hardware.

This assumes a `.env` file in the project root directory with the following content:
```
IBMQ_TOKEN=<IBM_TOKEN>
```

In [96]:
from dotenv import load_dotenv
from qiskit import IBMQ
import os
load_dotenv()

IBMQ.save_account(token=os.getenv("IBMQ_TOKEN"), overwrite=True)

## HHL Qiskit Implementation

### Initialization

Import necessary libraries and define constants:
* Import QuantumRegister and QuantumCircuit from Qiskit.
* Import numpy to perform mathematical operations.
* Define constants, such as the number of qubits, angle θ for `|b⟩ = [cos(θ), sin(θ)]`, and elements of the matrix A.
   * In this implementation, `A` is a [tridiagonal symmetric matrix](https://en.wikipedia.org/wiki/Tridiagonal_matrix).

In [97]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np

# Define constants
t = 2  # Time parameter for encoding the matrix A
NUM_QUBITS = 4  # Total number of qubits
nb = 1  # Number of qubits representing the solution
nl = 2  # Number of qubits representing the eigenvalues

theta = .75  # Angle defining |b>
a = 1  # Matrix diagonal
b = -1/3  # Matrix off-diagonal

Initialize quantum circuit and registers:
* Create a quantum register `qr` with the total number of qubits (`NUM_QUBITS`).
* Define the quantum register partitions (`qrb`, `qrl`, `qra`) for the solution, eigenvalue, and ancilla qubits respectively.
* Initialize a quantum circuit `qc` with the quantum registers `qr`.

In [98]:
# Initialize the quantum and classical registers
qr = QuantumRegister(NUM_QUBITS)

# Create a Quantum Circuit
qc = QuantumCircuit(qr)

# Define the quantum register partitions
qrb = qr[0:nb]  # Solution qubits
qrl = qr[nb:nb+nl]  # Eigenvalue qubits
qra = qr[nb+nl:nb+nl+1]  # Ancilla qubits

Apply a rotation (RY) gate on the first qubit (`qrb[0]`) with an angle of `2*θ` to prepare the state `|b⟩`.

In [99]:
# State preparation
qc.ry(2*theta, qrb[0])

### Quantum Phase Estimation

Performs QPE to estimate the eigenvalues of the matrix `A` encoded as a unitary operator `e^(iAt)`.

In [100]:
for qu in qrl:
    qc.h(qu)

qc.p(a*t, qrl[0])
qc.p(a*t*2, qrl[1])

qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])

In [101]:
# Controlled e^{iAt} on \lambda_{1}:
params=b*t

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

In [102]:
# Controlled e^{2iAt} on \lambda_{2}:
params = b*t*2

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

### Inverse Quantum Fourier Transform

In [103]:
qc.h(qrl[1])
qc.rz(-np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(-np.pi/4,qrl[0])
qc.h(qrl[0])

### Eigenvalue Rotation

This step is less of a "computation" step, and more of a "post-processing" step to extract the actual solutions from the eigenvalues. To make use of the estimated eigenvalues, we need to rotate them to a basis that is aligned with the computational basis. Directly below, t1-t4 calculate the rotational angles for the eigenvalues. The t values could be dynamically coded, but since we are only dealing with a 2x2 matrix, it makes sense to hard code them.

After calculating the rotation values, we apply a controlled-X gate to the eigenvalue and ancilla qubits to induce entanglement between the two. After, we apply a rotation (t1-t4) along the Y-axis to the Ancilla qubits.

In [ ]:
# Calculate Rotation Angles
t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

# Encode Eigenvalues into Ancilla Qubit
qc.cx(qrl[1],qra[0])
qc.ry(t1,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t2,qra[0])
qc.cx(qrl[1],qra[0])
qc.ry(t3,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t4,qra[0])

In [105]:
print(f"Depth: {qc.depth()}")
print(f"CNOTS: {qc.count_ops()['cx']}")
qc.draw(fold=-1)

Depth: 25
CNOTS: 10


┌─────────┐┌──────────────────────┐┌────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌─────────┐┌────────┐┌───┐┌─────────────┐┌───┐┌────────────┐┌─────────┐                                                                                                              
q40_0: ┤ Ry(1.5) ├┤ U(-0.66667,-π/2,π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(-0.66667) ├┤ X ├┤ Ry(0.66667) ├┤ P(3π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(-1.3333) ├┤ X ├┤ Ry(1.3333) ├┤ P(3π/2) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────
       └──┬───┬──┘└───────┬──────┬───────┘└────────┘└─┬─┘└──────────────┘└─┬─┘└─────────────┘└─────────┘└────────┘└─┬─┘└─────────────┘└─┬─┘└────────────┘└─────────┘                      ┌──────────┐     ┌───┐                                                                  
q40_1: ───┤ H ├───────────┤ P(2) ├────────────────────■────────────────────■────────────────────────────────────────┼───────────────────┼──────────────────────────────■───────────────■──┤ Rz(-π/4) ├─────┤ H ├────────■────────────────────────────────────────■────────────────
          ├───┤           ├──────┤                                                                                  │                   │      ┌───┐     ┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐└──────────┘     └───┘        │                                        │                
q40_2: ───┤ H ├───────────┤ P(4) ├──────────────────────────────────────────────────────────────────────────────────■───────────────────■──────┤ H ├─────┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ X ├─────■────────────────────────┼────────────────────■───────────────────┼────────────────
          └───┘           └──────┘                                                                                                             └───┘     └──────────┘└───┘└─────────┘└───┘   ┌─┴─┐    ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌─┴─┐┌────────────┐
q40_3: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Ry(-0.69352) ├┤ X ├┤ Ry(-0.87728) ├┤ X ├┤ Ry(0.35368) ├┤ X ├┤ Ry(1.2171) ├
                                                                                                                                                                                             └───┘    └──────────────┘└───┘└──────────────┘└───┘└─────────────┘└───┘└────────────┘

## Solution Measurement

In [106]:
from qiskit import ClassicalRegister

cr = ClassicalRegister(nb) 
meas = QuantumCircuit(qr, cr)
meas.measure(0, 0)

qc_with_meas = qc.compose(meas)

## Run on Quantum Hardware

Transpile the quantum circuit for running on IBM hardware.

In [107]:
from qiskit import IBMQ, transpile

provider = IBMQ.load_account()

backend = provider.get_backend('ibmq_quito') # calibrate using real hardware
layout = [2,3,0,4]
chip_qubits = 5

# Transpiled circuit for the real hardware
qc_with_meas_transpiled = transpile(qc_with_meas, backend=backend, initial_layout=layout)

ibmqfactory.load_account:WARNING:2023-04-25 20:23:38,659: Credentials are already in use. The existing account in the session will be replaced.


Run the job on the IBM quantum hardware.

In [108]:
NUM_SHOTS = 1024
result = backend.run(qc_with_meas_transpiled, shots=NUM_SHOTS).result()
counts = result.get_counts()

## Print the Results

In [109]:
print(counts)

{'0': 573, '1': 451}


In [110]:
# Estimate the amplitudes from the measurement statistics
prob_0 = counts.get('0', 0) / NUM_SHOTS
prob_1 = counts.get('1', 0) / NUM_SHOTS
alpha = np.sqrt(prob_0)
beta = np.sqrt(prob_1)

# Construct the approximate solution vector x
x = np.array([alpha, beta])

print(f"Quantum estimated solution vector: {x}")

Quantum estimated solution vector: [0.74804433 0.66364877]


## Compare to Classical Solution

In [111]:
A_numpy = np.array([[a, b],
              [b, a]])

b_numpy = np.array([np.cos(theta), np.sin(theta)])

x_classical = abs(np.linalg.solve(A_numpy, b_numpy))

x_classical_normalized = x_classical / np.linalg.norm(x_classical)

print(f"Classical solution vector: {x_classical_normalized}")

Classical solution vector: [0.71951437 0.69447755]


In [112]:
error = 100 * np.linalg.norm(x - x_classical_normalized) / np.linalg.norm(x_classical_normalized)
print(f"Percent error in quantum solution vector: {round(error, 5)}%")

Percent error in quantum solution vector: 4.20044%
